In [197]:
import numpy as np 
import cv2
from PIL import Image

In [198]:
prototxt_path = 'models/colorization_deploy_v2.prototxt'
model_path = 'models/colorization_release_v2.caffemodel'
kernel_path = 'models/pts_in_hull.npy'
image_path = 'lion.jpg'

In [199]:
end_size = 350
L_norm = 45

In [200]:
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
points = np.load(kernel_path)

In [201]:
points = points.transpose().reshape(2,313, 1, 1)
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
net.getLayer(class8).blobs = [points.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]
#net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]
#net.getLayer(net.getLayerId("conv8_313_rn")).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [202]:
bw_image = cv2.imread(image_path)
normalized_bw_image = bw_image.astype("float32") / 255.0
lab_image = cv2.cvtColor(normalized_bw_image, cv2.COLOR_BGR2LAB)
#model operuje na wymiarach 224x224 (na takich zostal wytrenowany)

In [203]:
resized = cv2.resize(lab_image, (224, 224))
L = cv2.split(resized)[0]
L -= L_norm

In [204]:
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

In [205]:
ab = cv2.resize(ab, (bw_image.shape[1], bw_image.shape[0]))
L = cv2.split(lab_image)[0]

In [206]:
colorized = np.concatenate((L[:,:,np.newaxis], ab), axis=2)
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = (255.0 * colorized).astype("uint8")

In [207]:

# cv2.imshow("BW Image", bw_image)
# cv2.imshow("Colorized", colorized)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [208]:
bw_image = cv2.resize(bw_image,(end_size,end_size))
colorized = cv2.resize(colorized,(end_size,end_size))
result = cv2.hconcat([bw_image,colorized])
cv2.imshow("Black and White -> Colour", result)
cv2.waitKey(0)

-1